# Propagation Project

In [2]:
import numpy as np

## 1. GEO to LEO

$$ P(A > A_{th}) = \sum P(A > A_{th} | \theta_i) P(\theta_i) $$



In [16]:
aus_37_5_50 = np.genfromtxt("./data/37_austria_5_50/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)
aus_37_55_90 = np.genfromtxt("./data/37_austria_55_90/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)
nor_37_5_50 = np.genfromtxt("./data/37_norway_5_50/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)
nor_37_55_90 = np.genfromtxt("./data/37_norway_55_90/output/ascii/attenuation_total.csv", delimiter=",", skip_header=7, filling_values=0)

print(aus_37_5_90[0:20, :])
aus_37_5_90 = np.concatenate((aus_37_5_50, aus_37_55_90))

[[1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 1.0000e-02
  1.5688e+02]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 1.6000e-02
  1.3949e+02]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 2.7000e-02
  1.2111e+02]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 4.5000e-02
  1.0459e+02]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 7.7000e-02
  8.9015e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 1.3000e-01
  7.5109e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 2.1000e-01
  6.3464e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 3.5000e-01
  5.2677e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 6.0000e-01
  4.3045e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 9.5000e-01
  3.6023e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 1.7000e+00
  2.8448e+01]
 [1.0000e+00 1.0000e+00 3.7500e+01 5.0000e+00 1.0000e+00 3.0000e+00
  2.2441e+01]
 [1.0000e+00 1.0